In [11]:
import pandas as pd

In [12]:
# Read file
df = pd.read_csv("hly175.csv", skiprows=15,low_memory=False)
df.head()

,date,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,msl
0,16-aug-2003 01:00,0,0.0,0,9.2,0,8.9,8.5,11.1,95,1021.9
1,16-aug-2003 02:00,0,0.0,0,9.0,0,8.7,8.5,11.1,96,1021.7
2,16-aug-2003 03:00,0,0.0,0,8.2,0,8.0,7.7,10.5,96,1021.2
3,16-aug-2003 04:00,0,0.0,0,8.4,0,8.1,7.9,10.7,97,1021.2
4,16-aug-2003 05:00,0,0.0,0,7.7,0,7.5,7.3,10.2,97,1021.1


In [14]:
# Add a column which takes the station name as value
df["station"]="PHOENIX PARK"
filename='PHOENIX PARK'
df.to_csv(f"{filename}.csv")
